In [1]:
!pip install einops pytorch_lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.3/812.3 kB 7.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.5/239.5 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.6/301.6 kB 14.6 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping

from einops import rearrange, repeat
from einops.layers.torch import Rearrange

In [3]:
class FFN(nn.Module):
    def __init__(self, dim, ffn_dim, dropout = 0.1):
        super(FFN, self).__init__()
        self.net = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, ffn_dim),
            nn.GELU(),
            nn.Linear(ffn_dim, dim),
            nn.LayerNorm(dim),
            nn.Dropout(dropout)
        )
    def forward(self, x):
        return self.net(x)

class Embeddings(nn.Module):
    def __init__(self, patch_size, img_size, dim = 128, channels = 3):
        super(Embeddings, self).__init__()
        assert img_size % patch_size == 0, "image size should be able to be divided by patch size"
        patches = (img_size // patch_size) ** 2
        patch_dim = patch_size ** 2 * channels

        self.to_dim = nn.Sequential(
            Rearrange('b c (h p1) (w p2) -> b (h w) (c p1 p2)', p1=patch_size, p2=patch_size),
            nn.LayerNorm(patch_dim),
            nn.Linear(patch_dim, dim),
            nn.LayerNorm(dim)
        )

        self.pos_emb = nn.Parameter(torch.randn(1, patches, dim))

    def forward(self, x):
        x = self.to_dim(x)
        x += self.pos_emb
        return x

class Attention(nn.Module):
    def __init__(self, dim, heads=8, dim_heads=64, dropout=0.1):
        super(Attention, self).__init__()

        inner_dim = heads * dim_heads

        self.heads = heads
        self.toqkv = nn.Linear(dim, inner_dim * 3)

        self.flash = dict(
            enable_flash=True,
            enable_math=True,
            enable_mem_efficient=True
        )

        self.to_out = nn.Sequential(
            nn.Linear(inner_dim, dim),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        qkv = self.toqkv(x).chunk(3, dim=-1)
        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h=self.heads), qkv)

        with torch.backends.cuda.sdp_kernel(**self.flash):
            out = F.scaled_dot_product_attention(q, k, v)

        out = rearrange(out, 'b h n d -> b n (h d)')
        out = self.to_out(out)

        return out


In [4]:
class ViT(nn.Module):
    def __init__(self, patch_size, img_size, depth, classes, heads=8, dim=128, ff_dim=128, channels=3, dim_heads=64, dropout=0.1):
        super(ViT, self).__init__()
        self.embedding = Embeddings(patch_size, img_size, dim, channels)
        self.layers = nn.ModuleList([])
        for _ in range(depth):
            self.layers.append(nn.ModuleList([
                Attention(dim, heads, dim_heads, dropout),
                FFN(dim, ff_dim, dropout)
            ]))
        self.ln = nn.LayerNorm(dim)
        self.to_logits = nn.Linear(dim, classes)

    def forward(self, x):
        x = self.embedding(x)
        for attn, ffn in self.layers:
            x = attn(x) + x
            x = ffn(x) + x 
        x = self.ln(x)
        x = x.mean(dim=1)
        return self.to_logits(x)

In [5]:
class ViTModule(pl.LightningModule):
    def __init__(self, patch_size, img_size, depth, num_classes, heads=8, dim=128, ff_dim=128, channels=3, dim_heads=64, dropout=0.1):
        super(ViTModule, self).__init__()
        self.model = ViT(patch_size, img_size, depth, num_classes, heads, dim, ff_dim, channels, dim_heads, dropout)
        self.criterion = nn.CrossEntropyLoss()

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.criterion(logits, y)
        self.log('train_loss', loss, prog_bar=True)
        preds = torch.argmax(logits, dim=1)
        accuracy = (preds == y).float().mean()
        self.log('train_acc', accuracy, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.criterion(logits, y)
        self.log('val_loss', loss)
        preds = torch.argmax(logits, dim=1)
        accuracy = (preds == y).float().mean()
        self.log('val_acc', accuracy, prog_bar=True)
        return loss

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=1e-4)
        return optimizer

# Assuming you have data loaders for CIFAR-100
transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)),
])

batch_size = 2048

train_dataset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# Initialize ViT model
patch_size = 4
img_size = 32
depth = 6
num_classes = 100

vit_model = ViTModule(patch_size, img_size, depth, num_classes)

# Initialize a trainer
trainer = pl.Trainer(max_epochs=100, callbacks = EarlyStopping(monitor="val_acc", min_delta=0.00, patience=5, verbose=False, mode="max"))

# Train the model
trainer.fit(vit_model, train_loader, test_loader)

100%|██████████| 169001437/169001437 [00:09<00:00, 17665044.05it/s]


Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
You are using a CUDA device ('NVIDIA RTX A4000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Mis

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (25) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]